# Hybrid Search with Elasticsearch and local embeddings

## Preconditions

To make this notebook work, you need to have an elasticsearch service (> `v8.9.0`) running on [http://localhost:9200](http://localhost:9200). To use the Reciprocal Rank Fusion in the byrid search (in the end of the notbook), you need to have a Platinum/Enterprise subscription [https://www.elastic.co/subscriptions](https://www.elastic.co/subscriptions)


## Helper Methods

In [1]:
def show_chunks(docs):
    for doc in docs:
        # check if doc is tuple
        if isinstance(doc, tuple):
            doc, score = doc
        else:
            score = None
        print("Page", doc.metadata["page"], ":")
        print(doc.page_content.replace('\n', ' '))
        if score:
            print("Score:", score)
        print(100 * "-")


## Load and chunk Documents

In [2]:
from pathlib import Path

raw_data_file = Path.cwd() / "sbb-geschaeftsbericht-2023.pdf"

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader(str(raw_data_file))
chunks = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0))
print("total chunks: ", len(chunks))
print('-' * 100)
print("Example:")
chunks[25]


total chunks:  970
----------------------------------------------------------------------------------------------------
Example:


Document(page_content='Geschäftsbericht 20238Konzernlagebericht\nDas Jahr 2023 in Zahlen. \n11,26\xa0Milliarden Franken \nSchulden Schuldendeckungsgrad: 7,82\n2700\xa0Sendungen Ein weiterer Anstieg der Verschuldung konnte gestoppt \nwerden (2022: 11,44\xa0Milliarden Franken). Doch der \nSpardruck bleibt hoch. Für eine nachhaltige Finanzie -\nrung wird die SBB  bis 2030 rund 6\xa0Milliarden Franken \nweniger ausgeben.Der Schuldendeckungsgrad misst, wie viele Jahre das laufende Ergebnis erzielt werden müsste, um die', metadata={'source': '/home/ursin/development/hybrid-search-demo/sbb-geschaeftsbericht-2023.pdf', 'page': 7})

## Create Embeddings using an Opensource Embedding Model

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

e5_multilingual_embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
e5_multilingual_embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='intfloat/multilingual-e5-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

## Create Embedding from Question

In [5]:
query = "Wie viel weniger will die SBB bis 2030 ausgeben?"
e5_multilingual_embedding_model.embed_query(query)

[-0.020724598318338394,
 0.003353245323523879,
 -0.04718921333551407,
 -0.04045533016324043,
 0.026072410866618156,
 -0.018294258043169975,
 -0.01445701438933611,
 0.02227230742573738,
 0.06754158437252045,
 0.001783400890417397,
 0.0425213947892189,
 0.030309047549962997,
 -0.012533361092209816,
 -0.02684326469898224,
 -0.026066487655043602,
 -0.05336495116353035,
 0.023507127538323402,
 0.00879379641264677,
 -0.012215628288686275,
 -0.020137257874011993,
 0.03075406886637211,
 -0.04414905235171318,
 -0.01693112961947918,
 -0.000751925224903971,
 -0.007161107379943132,
 0.011271339841187,
 -0.032673366367816925,
 -0.019964460283517838,
 -0.007265779655426741,
 0.01016951072961092,
 -0.0591048002243042,
 -0.0019779279828071594,
 -0.043426934629678726,
 -0.020695149898529053,
 -0.020205454900860786,
 0.011324039660394192,
 0.05723607540130615,
 0.029726767912507057,
 -0.04964124783873558,
 0.01168445497751236,
 -0.027545282617211342,
 0.026116862893104553,
 -0.003331550629809499,
 -0.01

## Setup Connection to Elasticsearch

In [6]:
from langchain_elasticsearch import ElasticsearchStore

es_vector_search = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="test_index_vector_only",
    embedding=e5_multilingual_embedding_model
)

es_vector_search.add_documents(chunks)

['d13c278e-e5c8-4e5d-a5b7-55a3a1460402',
 '7b679e41-10f5-4020-b6da-5c9563c5afab',
 '48ec5081-84a2-49aa-8dc8-6ab8e0e4d9c9',
 '2c479fd4-cb5f-4bb4-95e3-c9bc12b0e517',
 '23b5af84-5129-44bb-904e-657625bf11cb',
 '65ec5af7-fae5-4bd0-bfc4-afed4ab263d1',
 '65c85b9d-4161-4c3a-acce-54e66930f801',
 '60dff54a-30fd-4301-ab91-73da7a7bc9ca',
 '5157c7ac-77b3-47d8-9fe4-08ee93ec62f7',
 '0863c0d9-7483-4e00-ad14-0b5cbded875b',
 '67f028ec-d6d3-4979-be9a-23e965475cfe',
 '22d76921-a49a-48d0-9079-fb0e48e0e6da',
 '1dab5a95-b00c-434b-b382-3a3f7d6886d5',
 '4feaec11-92a5-42b8-afbc-de9e63791b5c',
 '73c723e6-020a-4e7d-875d-b1196d2741c7',
 '6d04ef86-4854-4368-b916-d2be66d1b9e5',
 '242f5367-c30c-45e9-9142-6651450944f9',
 '91fb10ac-bbcb-4703-bd8a-64042c748b6c',
 '5d5503f6-bbf4-4dcf-8486-feeaf85ecf00',
 '2387e028-cacf-4268-a359-231a59a0a2c9',
 'b45f5d4a-6c81-4677-bb05-402f804430f1',
 '5db47657-ba77-4db5-bc88-913f5018879c',
 'caa19ff6-3e1b-4147-9931-77fd58986024',
 'c96181c4-5dc4-4e4a-b384-5fdbcb1fe512',
 'd090a446-40d4-

## Search with native Elasticsearch similarity search

In [7]:
similar_documents = es_vector_search.similarity_search_with_score(query, k=4)
show_chunks(similar_documents)

Page 45 :
ten. Trotz eines positiven Konzernergebnisses bleibt der Spar- und Effizienzdruck hoch. Das positive Jahresergeb - nis reicht noch nicht aus, um den notwendigen Schulden - abbau zu erreichen. Für eine nachhaltig finanzierte  SBB  sind jährlich rund 500 Millionen Franken Gewinn nötig. Nur  so können die Schulden abgebaut und anstehende In - vestitionen, unter anderem in neues Rollmaterial und in  Bahnhöfe, getätigt werden. Stabilisierungspaket – nachhaltige   Finanzierung bis 2030.
Score: 0.93489087
----------------------------------------------------------------------------------------------------
Page 45 :
Gesunde Finanzen sind für die Zukunft der SBB  essen - ziell. Das Verschuldungsniveau muss gesenkt werden  können, die Zinslast tragbar sein und das Eignerziel zum Schuldendeckungsgrad erreicht werden. Um dies zu er - wirken, leistet die  SBB  selbst einen grossen Beitrag, in - dem sie bis 2030 rund sechs Milliarden Franken weniger  ausgibt. Erträge aus Immobilien sichern 

## Search with a hybrid approach
To understand this approach better, have a look at: https://learn.microsoft.com/en-us/azure/search/hybrid-search-ranking


In [8]:
es_hybrid_search = ElasticsearchStore.from_documents(
    chunks, 
    e5_multilingual_embedding_model, 
    es_url="http://localhost:9200", 
    index_name="test_index_hybrid",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        hybrid=True,
    )
)

The following cell will only work if you have a paid ES version with supports Reciprocal Rank Fusion (RRF). Alterantively, you can build it yourself for example with the langchain **EnsembleRetriever**. Have a look at [https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

In [9]:
similar_documents = es_hybrid_search.similarity_search(query, k=3)
show_chunks(similar_documents)

AuthorizationException: AuthorizationException(403, 'security_exception', 'current license is non-compliant for [Reciprocal Rank Fusion (RRF)]')